
# Inference of Mass Eruption Rate from a single observation of Plume Height

In this example, we illustrate how MERPH can be used to generate plausible estimates of the Mass Eruption Rate (MER) from an observation of the plume height (H) using Mastin's data set.

We will import the data and take a quick look at it.  We will then specify the independent (explanatory) and dependent (output) variables, and set an observation.  Estimates will be generated by drawing samples from the posterior predictive distribution.


<div class="alert alert-info"><h4>Note</h4><p>This example can be viewed as a Jupyter notebook, launched from the commandline using `merph --example 1` or from an interactive python session using

```python
import merph
merph.launch_jupyter_example(1)</p></div>
```
## Imports

First, we import MERPH package (and matplotlib for plotting)



In [ ]:
import matplotlib.pyplot as plt

import merph

## Exploratory data analysis

The MERPH packages includes four data sets; those of Sparks, Mastin, and Aubry, and the IVESPA data.  Here we will use Mastin's data:



In [ ]:
Mastin = merph.load_Mastin()

The object `Mastin` is an instance of the `Merph` class within merph.  We can learn about it by printing:



In [ ]:
print(Mastin)

The `Merph` object, `Mastin`, contains the data in the Mastin dataset and methods to perform statistical analysis on this data.

We can view the data set as a pandas dataframe using `Mastin.data`.  Additionally, methods for the pandas dataframe can be called.  For example, we can plot the data using tools from `pandas.DataFrame.plot`.

As an example, we can produce a scatter plot the mass eruption rate (contained in the 'MER' column of the dataframe) and the plume height (contained in the 'Plume height' column):



In [ ]:
_ = Mastin.data.plot.scatter(x='MER', y='Plume height')

Interesting... but the linear scale for the MER-axis is not helping here.  Perhaps we should re-plot with a logarithmic scale for MER:



In [ ]:
_ = Mastin.data.plot.scatter(x='MER', y='Plume height', logx=True)

Now a very clear relationship appears.  The plume height increases with the MER, and it looks like a power-law relationship would describe this, i.e. $H = a \times Q^{b}$ for some values of $a$ and $b$.

This power-law relationship is expected from dimensional reasoning and from theoretical analysis of the fluid mechanics for turbulent plumes.

We can find the values of $a$ and $b$ using linear regression of logarithmically transformed data.  Taking logarithms we find a linear relationship, $\log H = \log a + b\log Q$, so finding the slope and intercept of the data on a log-log plot would give a prediction of connection between plume height and mass eruption rate.  (The plot below shows Mastin's data on log-log scales, and a straight-line fit looks like it would be reasonable.)




In [ ]:
_ = Mastin.data.plot.scatter(x='MER', y='Plume height', logx=True, logy=True)

In MERPH, we use the logarithmic transformation of the data to look for the relationship between the plume height and mass eruption rate.  For the Mastin data loaded with `merph.load_Mastin()`, the columns in the dataframe corresponding the plume height and mass eruption rate have been assigned, and can be found using `height_column` and `mer_column`` attributes of the `Merph`</code>` object.



In [ ]:
print(f"MER data is in the column called '{Mastin.mer_column}'")
print(f"Plume height data is in the column called '{Mastin.height_column}'")

## Linear regression by maximum likelihood estimation

To perform regression analysis using the Mastin data, we need to specify the 'explanatory' (independent) and 'response' (dependent) variables.  These are often labelled as the $x$ and $y$ variables, and we use this in merph for simplicity.

Most commonly we will want to predict the mass eruption rate from an observation of the plume height.  Thus, we have the $x$ variable as logarithm of the plume height ($x = \log_{10} H$), and the $y$ variable as the logarithm of the mass eruption rate ($y = \log_{10} Q$).

We specify this for the `Merph` object using the `set_vars` method, where we specify `xvar` and `yvar` as either `'H'` for plume height or `'Q'` (or `'MER'`) for the mass eruption rate.

Here we want plume height (`'H'`) to be the explanatory variable, and predict the mass eruption rate (`'Q'`) from it, so we specify:



In [ ]:
Mastin.set_vars(xvar='H', yvar='Q')

We can then proceed to perform the linear regression of the log-transformed data.  In merph this is done using maximum likelihood estimation, implemented automatically when the variables are set.

This produces the posterior distribution for the model parameters.

We can visualize this using the `plot` method of the `posterior` subclass.



In [ ]:
# sphinx_gallery_thumbnail_number = 6
fig, ax = Mastin.posterior.plot()

The maximum likelihood estimator of the linear regression coefficients in the log-log space produces a nice fit, and gives a good fit of the data in the semi-log space.

## Bayesian linear regression

While the linear regression fits the trend of the data, there remains considerable scatter around the curve.  The amount of scatter can be substantial.  For example, for a plume height of around 10 km, the data spans mass eruption rates between $\sim 5\times 10^{5}$ and $\sim 5\times 10^{7}$ -- two orders of magnitude.

To make meaningful inference of the mass eruption rate from the plume height, it is important to consider the scatter.  With maximum likelihood estimation, we can provide confidence intervals, but a Bayesian approach to linear regression provides other advantages.

In Bayesian linear regression, we propose a model

\begin{align}y_{i} = \beta_{0} + \beta_{1}x_{i} + \epsilon_{i},\end{align}

(recalling $y_{i}=\log_{10} Q_{i}$, $x_{i} = \log_{10} H_{i}$, so $Q=10^{\beta_{0}} H^{\beta_{1}}$) where $\epsilon_{i}$ is a model error term that is included to account for deviation from the straight-line fit.  If we assume the errors are independent and identically distributed, then the error can be modelled as $\epsilon_{i} \sim N(0, \sigma^{2})$, with a constant variance $\sigma^{2}$.  Our goal is to determine the model parameters $\mathbf{\beta} = (\beta_{0},\beta_{1})$ and $\sigma^{2}$.

In Bayesian linear regression, we specify priors on the model parameters, and it is common to take a non-informative prior of the form

\begin{align}p(\mathbf{\beta},\sigma^{2}) \propto 1/\sigma^{2}.\end{align}

Posterior distributions for the model parameters are found using Bayes' theorem, using the data and prior.

In merph, the inference of the model parameters is performed when the variables are specified (i.e. when the `set_vars()` method is called for a dataset).

The posterior distributions can be explored using the methods in the `posterior` subclass.  Perhaps the most useful is the visualization of curve fits obtained from sampling the posterior distributions:




In [ ]:
fig, ax = Mastin.posterior.plot(samples=1000,interval=[0.9,0.95,0.99])

Here, we have sampled 1000 plausible line fits from the posterior distribution, and plotted these together with three credible intervals (at the 90%, 95% and 99% levels), the mean curve and the data.

We can use these to make predictions of the response variable given an observation of the explanatory variable while accounting for the uncertainty in the dataset.  This is known as *Bayesian posterior prediction*.

## Bayesian posterior prediction for mass eruption rate

The posterior predictive distribution can be visualized using the method `posterior_plot()` on the `Merph` object.



In [ ]:
fig, ax = Mastin.predictive_plot()

Here we see the observed data and the regression curve found by maximum likelihood estimation.  Additionally, for each plume height, we determine the posterior predictive distribution of the mass eruption rate, and plot prediction intervals.  The contours provide an appropriate envelope of the observations, illustrating the statistical model is appropriate.

Suppose we have an observation of the plume height (in this example taken to be precise) of $H = 10$ km.  We can make predictions of the MER from the model using the `posterior_predictive` subclass, specifing this observation.  Here we store this in an instance of the subclass called `pred`:



In [ ]:
pred = Mastin.posterior_predictive([10,20.])

.. important::

    The observation here is given in the units of the data (in this case, height in km).  It is possible to use log-transformed values by passing the optional argument `logscale=True` to `posterior_predictive`  

    This will then produce predictions for the response variable as log-transformed values.


We can then draw an estimate of the mass eruption rate from the posterior predictive distribution using the `simulate()` method of the `pred` object.  This produces a DataFrame with the observation and samples of the response variable, as well as the logarithm of these (which are used directly in the statistical model).  To draw five samples:



In [ ]:
pred.simulate(5)

<div class="alert alert-info"><h4>Note</h4><p>As the samples are determined stochastically, the values obtained will be different each time `simulate` is called.</p></div>

<div class="alert alert-info"><h4>Note</h4><p>The DataFrame has ten rows as `simulate(5)` computes five samples for each of the discrete observations passed to `posterior_predictive` (here there are two observed heights).</p></div>

Running `pred.simulate()` again will produce a different estimate, as we draw model parameters (including the error term) from their posterior distributions.

Passing the option `plot=True` to `posterior_predictive` creates a histogram of the posterior predictions.  Here we draw 1000 samples and view the histogram.



In [ ]:
posterior_samples = pred.simulate(1000, plot=True)

The plot shows histogram two histograms for the posterior predictive samples.

 The `posterior_predictive` subclass also contains methods that determine other useful quantities from the posterior predictive distribution. 

 Here we illustrate them using the `pred` instance.

- `pred.rvs(N)` draws `N` samples from the posterior predictive distribution, returning a numpy array.



In [ ]:
pred.rvs(5)

- `pred.pdf(Q)` computes the probability density of $Q|H$, returning a numpy array.  The argument `Q` can be a single value or an array of values.



In [ ]:
pred.pdf([1e6,5e6,1e7])

.. important::

        The posterior probability densities integrate to one, so the pdf at values of the MER are small, since the MER varies over several orders of magnitude.

- `pred.cdf(Q)` computes the cumulative density probability of $Q|H$ returning a numpy array.  This is the probability $P(MER < Q | H)$.  The argument `Q` can be a single value or an array of values.



In [ ]:
pred.cdf([1e6,5e6,1e7])

<div class="alert alert-info"><h4>Note</h4><p>The cdf can be used to find the probability that the response variable takes values in a range.

        Denoting the posterior predictive cdf as $F_{Q|H}(q) = P(Q\leq q|H)$ we have

            .. math::

                P(q_{0}\leq Q \leq q_{1} | H) = F_{Q|H}(q_{1}) - F_{Q|H}(q_{0})</p></div>

- `pred.sf(Q)` computes the survival function probability of $Q|H$, returning a numpy array.  This is the probability $P(MER > Q | H)$.  The argument `Q` can be a single value or an array of values.



In [ ]:
pred.sf([1e6,5e6,1e7])

<div class="alert alert-info"><h4>Note</h4><p>The survival function gives $P(Q\geq q|H) = 1 - F_{Q|H}(q)$.</p></div>

- `pred.ppf(alpha)` computes the probability point function of $Q|H$, returning a numpy array. The argument alpha can be a single value or an array of values.



In [ ]:
pred.ppf([0.75, 0.9])

<div class="alert alert-info"><h4>Note</h4><p>The probability point function is the inverse of the cumulative distribution function, so allows us to find the value $Q$ such that $P(MER \leq Q | H) = \alpha$.

    As an example, we can compute the median by setting $\alpha = 0.5$</p></div>



In [ ]:
print(f"The median of the posterior predictive distribution for MER | H = {pred.obs[0]} km is {pred.ppf(0.5)[0][0]:.2e} kg/s")

- `pred.isf(alpha)` computes the inverse survival function of $Q|H$, returning a numpy array. The argument alpha can be a single value or an array of values.



In [ ]:
pred.isf([0.75, 0.9])

<div class="alert alert-info"><h4>Note</h4><p>The inverse survival function allows us to find the value $Q$ such that $P(MER \geq Q | H) = \alpha$.

    As an example, we can compute the median by setting $\alpha = 0.5$</p></div>



In [ ]:
print(f"The median of the posterior predictive distribution for MER | H = {pred.obs[0]} km is {pred.isf(0.5)[0][0]:.2e} kg/s")

- `pred.median()` computes the median of the posterior predictive distribution, returning a numpy array.



In [ ]:
print(f"The median of the posterior predictive distribution for MER | H = {pred.obs[0]} km is {pred.median()[0]:.2e} kg/s")

- `pred.mean()` computes the mean of the posterior predictive distribution, returning a numpy array.



In [ ]:
print(f"The mean of the posterior predictive distribution for log MER | H = {pred.obs[0]} km is {pred.mean()[0]:.3f}")

- `pred.std()` computes the standard deviation of the posterior predictive distribution, returning a numpy array.



In [ ]:
print(f"The standard deviation of the posterior predictive distribution for log MER | H = {pred.obs[0]} km is {pred.std()[0]:.3f}")

- `pred.var()` computes the variance of the posterior predictive distribution, returning a numpy array.



In [ ]:
print(f"The variance of the posterior predictive distribution for log MER | H = {pred.obs[0]} km is {pred.var()[0]:.3f}")

.. important::

        The mean, standard deviation and variance of the posterior distribution function are only defined in log-space.  The results are therefore in log-space.

- `pred.interval(alpha)` computes credible intervals with equal areas around the median of the posterior predictive distribution, returning a numpy array.  As an example, a 95% credible prediction interval can be computed by setting $\alpha = 0.95$



In [ ]:
intervals = pred.interval(0.95)[0,:,:]
print(f"The 95% credible interval for MER | H = {pred.obs[0]} km is [{intervals[0,0]:.2e},{intervals[1,0]:.2e}] kg/s")

There is also a `plot` subclass of `posterior_predictive` that produces plots from some of the quantities.  The use is illustrated below.

The posterior predictive density can be plotted using `plot.pdf()`, returning a Figure and Axes instance.



In [ ]:
fig, ax = pred.plot.pdf()

The posterior predictive cumulative density can be plotted using `plot.cdf()`, returning a Figure and Axes instance.



In [ ]:
fig, ax = pred.plot.cdf()

The posterior predictive probability point function can be plotted using `plot.ppf()`, returning a Figure and Axes instance.



In [ ]:
fig, ax = pred.plot.ppf()